
## Momentum Score Implementation

Link - https://www.msci.com/eqb/methodology/meth_docs/MSCI_Momentum_Indexes_Methodology_Aug2021.pdf

In [254]:
# Important libraries used

import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt
import time
import pickle

# To diable yahoo finance logging processing messages
import logging
logger = logging.getLogger('yfinance')
logger.disabled = True
logger.propagate = False

### Universe
- Nifty 500
- S&P 500

In [255]:
# Uploading list of tickers for S&P500 and Nifty500 - Universe

nf500 = pd.read_csv("ind_nifty500list.csv")
sp500 = pd.read_csv("sp500_companies.csv")

In [256]:
# Checking columns
nf500.columns, sp500.columns

(Index(['Company Name', 'Industry', 'Symbol', 'Series', 'ISIN Code'], dtype='object'),
 Index(['Exchange', 'Symbol', 'Shortname', 'Longname', 'Sector', 'Industry',
        'Currentprice', 'Marketcap', 'Ebitda', 'Revenuegrowth', 'City', 'State',
        'Country', 'Fulltimeemployees', 'Longbusinesssummary', 'Weight'],
       dtype='object'))

In [257]:
# Check that market cap is sorted for S&P 500
print(sum(sp500.Marketcap - sp500.Marketcap.sort_values()))
print(nf500.shape)

0
(503, 5)


In [258]:
# Seperating tickers for use

nf_tickers = list(nf500.Symbol)
sp_tickers = list(sp500.Symbol)

sp_ohlcv = {}
nf_ohlcv = {}

### Download Data

Based on the MSCI paper we need data for a time period of 1 week i.e. 5 days.

Using a lookback of 10 years.

### BE AWARE OF DOWNLOAD CALL CELLS THEY TAKE TIME TO FETCH
#### MARKED WITH "CAUTION"

In [259]:
# Download data with time delay

def download_data(ohlcv, tickers, start, end, period = "5d", region="", left=0, right=-1):
    idx = 0
    for ticker in tickers[left:right]:
        ohlcv[ticker] = yf.download(ticker +region, start, end, period = period).loc[:, ["Adj Close"]]
        print(left+idx)
        idx += 1
        # time.sleep(2) # Added sleep time not to overload the server

In [260]:
# Nifty 500 Data

start = dt.datetime.today() - dt.timedelta(3650)
end = dt.datetime.today()



# Use the pickle files provided with the code instead to load data Uncomment the code below
###############################################

with open("nifty_500.pickle" , 'rb') as f:
    nf_ohlcv = pickle.load(f)

###############################################


# CAUTION
# This step will take 25 mins approx Run separately or together based on time
# rl_ohlcv = {}
# download_data(rl_ohlcv, ["360ONE"], start=start, end=end, period="5d", region=".NS", left = 0, right=1)
# download_data(nf_ohlcv, nf_tickers, start=start, end=end, period="5d", region=".NS", left = 0, right=100)

# download_data(nf_ohlcv, nf_tickers, start=start, end=end, period="5d", region=".NS", left = 100, right = 200)

# download_data(nf_ohlcv, nf_tickers, start=start, end=end, period="5d", region=".NS", left = 200, right = 300)

# download_data(nf_ohlcv, nf_tickers, start=start, end=end, period="5d", region=".NS", left = 300, right = 400)

# download_data(nf_ohlcv, nf_tickers, start=start, end=end, period="5d", region=".NS", left = 400, right = -1)



In [261]:
# Remove empty dataframe and save the data to pickle file

empty = []
for ticker in nf_tickers:
    if ticker in nf_ohlcv:
        if nf_ohlcv[ticker].shape[0]==0:
            empty.append(ticker)
    else:
        empty.append(ticker)
        
print(empty)
for ticker in empty:
    nf_ohlcv.pop(ticker, None)
    nf_tickers.remove(ticker)

print(len(nf_ohlcv.keys()), len(nf_tickers))

# Saving data to pickle file to avoid re-download
# with open('nifty_500.pickle', 'wb') as f:
#     pickle.dump(nf_ohlcv, f)

['DUMMYRAYMD', 'DUMMYSANOF']
501 501


In [262]:
# S&P 500 Data


# Use the pickle files provided with the code instead to load data Uncomment the code below
############################################

with open("sp_500.pickle" , 'rb') as f:
    sp_ohlcv = pickle.load(f)

############################################


# CAUTION
# This step will take 15 mins approx Run separately or together based on time - Also uncomment the save code in next code block

# download_data(sp_ohlcv, sp_tickers, start=start, end=end, period="5d", region="", left=0, right=100)

# download_data(sp_ohlcv, sp_tickers, start=start, end=end, period="5d", region="", left=100, right=200)

# download_data(sp_ohlcv, sp_tickers, start=start, end=end, period="5d", region="", left=200, right=300)

# download_data(sp_ohlcv, sp_tickers, start=start, end=end, period="5d", region="", left=300, right=400)

# download_data(sp_ohlcv, sp_tickers, start=start, end=end, period="5d", region="", left=400, right=-1)

In [263]:
# Remove empty dataframe and save the data to pickle file

empty = []
for ticker in sp_tickers:
    if ticker in sp_ohlcv:
        if sp_ohlcv[ticker].shape[0]==0:
            empty.append(ticker)
    else:
        empty.append(ticker)
print(empty)
for ticker in empty:
    sp_ohlcv.pop(ticker, None)
    sp_tickers.remove(ticker)

print(len(sp_ohlcv.keys()), len(sp_tickers))


# Saving data to pickle file to avoid re-download
# with open('sp_500.pickle', 'wb') as f:
#     pickle.dump(sp_ohlcv, f)

['ETSY']
502 502


### Determination of Momentum Score

In [264]:
# Monthly Price Momentum
def price_momentum(data, period, tickers, local_risk_free_rate):
    for ticker in tickers:
        data[ticker][str(period)+"_mon_price_mom"] = (data[ticker]["Adj Close"].shift(int(252/52))/data[ticker]["Adj Close"].shift(int((period+1)*252/52))-1) - local_risk_free_rate


In [265]:
# 6 and 12-month price momentum # local risk free rate taken from 5 year treasury bond yield - INDIA and USA - Recommendation from Business Valuation by Damodaran

price_momentum(nf_ohlcv, period=6, tickers=nf_tickers, local_risk_free_rate=.06889)
price_momentum(nf_ohlcv, period=12, tickers=nf_tickers, local_risk_free_rate=.06889)

# 6 and 12-month price momentum # local risk free rate taken from 5 year treasury bond yield - INDIA and USA

price_momentum(sp_ohlcv, period=12, tickers=sp_tickers, local_risk_free_rate=.036)
price_momentum(sp_ohlcv, period=6, tickers=sp_tickers, local_risk_free_rate=.036)

In [266]:
# checking calculation for one stock

nf_ohlcv["360ONE"]

,Adj Close,6_mon_price_mom,12_mon_price_mom
Date,,,
2019-09-19,262.504547,NaN,NaN
2019-09-20,275.624512,NaN,NaN
2019-09-23,289.405792,NaN,NaN
2019-09-24,303.868958,NaN,NaN
2019-09-25,290.283875,NaN,NaN
...,...,...,...
2024-08-16,1069.599976,-0.011955,0.303417
2024-08-19,1098.050049,-0.002230,0.288499
2024-08-20,1098.750000,-0.002965,0.220411


### Risk-Adjusted Momentum Value

In [267]:
# First we need to calculate volatility for each for 7 day price and 3 year period
def volatility(data, tickers, window_size):
    for ticker in tickers:
        
        data[ticker]["7day_3year_sigma"] = data[ticker]["Adj Close"].pct_change().rolling(window_size).std()*(52**0.5)


In [268]:
# Calculating for both Nifty and S&P
# Window size is for 3 years

volatility(nf_ohlcv, tickers=nf_tickers, window_size=3*52)
volatility(sp_ohlcv, tickers=sp_tickers, window_size=3*52)

In [269]:
# Risk adjusted momentul for 6 and 12 months
def risk_adjusted_momentum(data, tickers, period):
    for ticker in tickers:
        data[ticker][str(period)+"_mon_risk_adj_mom"] = data[ticker][str(period)+"_mon_price_mom"]/data[ticker]["7day_3year_sigma"]

In [270]:
# 6 and 12 month adjustment for NIFTY500
risk_adjusted_momentum(nf_ohlcv, tickers=nf_tickers, period=6)
risk_adjusted_momentum(nf_ohlcv, tickers=nf_tickers, period=12)

# 6 and 12 month adjustment for S&P500
risk_adjusted_momentum(sp_ohlcv, tickers=sp_tickers, period=6)
risk_adjusted_momentum(sp_ohlcv, tickers=sp_tickers, period=12)

In [271]:
# Checking calculations so far

nf_ohlcv["360ONE"]

,Adj Close,6_mon_price_mom,12_mon_price_mom,7day_3year_sigma,6_mon_risk_adj_mom,12_mon_risk_adj_mom
Date,,,,,,
2019-09-19,262.504547,NaN,NaN,NaN,NaN,NaN
2019-09-20,275.624512,NaN,NaN,NaN,NaN,NaN
2019-09-23,289.405792,NaN,NaN,NaN,NaN,NaN
2019-09-24,303.868958,NaN,NaN,NaN,NaN,NaN
2019-09-25,290.283875,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2024-08-16,1069.599976,-0.011955,0.303417,0.179797,-0.066491,1.687559
2024-08-19,1098.050049,-0.002230,0.288499,0.179356,-0.012435,1.608531
2024-08-20,1098.750000,-0.002965,0.220411,0.177234,-0.016729,1.243617


# CHECKED ABOVE THIS

### Calculating Momentum Score

In [272]:
# Extract data from the individual DB into a merged db of all tickers
mon_6_nf  = {}
mon_12_nf  = {}

mon_6_sp  = {}
mon_12_sp  = {}

def mergeData(mon_6_db, mon_12_db, ohlcv, tickers):
    for ticker in tickers:
        mon_6_db[ticker] = ohlcv[ticker]["6_mon_risk_adj_mom"]
        mon_12_db[ticker] = ohlcv[ticker]["12_mon_risk_adj_mom"]

mergeData(mon_6_nf, mon_12_nf, nf_ohlcv, nf_tickers)
mergeData(mon_6_sp, mon_12_sp, sp_ohlcv, sp_tickers)

In [273]:
mon_6_combined_nf = pd.DataFrame.from_dict(mon_6_nf)
mon_12_combined_nf = pd.DataFrame.from_dict(mon_12_nf)

mon_6_combined_sp = pd.DataFrame.from_dict(mon_6_sp)
mon_12_combined_sp = pd.DataFrame.from_dict(mon_12_sp)

In [274]:
# Saving the calculations for safe keep and manual checking

# mon_6_combined_nf.to_csv("mon_6_combined_nf.csv")
# mon_12_combined_nf.to_csv("mon_12_combined_nf.csv")

# mon_6_combined_nf.to_csv("mon_6_combined_sp.csv")
# mon_12_combined_nf.to_csv("mon_12_combined_sp.csv")

In [275]:
# First we need to calculate mean and std for each row and save in the respective columns

mon_6_combined_nf["mean"] = list(mon_6_combined_nf.mean(axis=1))
mon_6_combined_nf["std"] = list(mon_6_combined_nf.std(axis=1))

mon_12_combined_nf["mean"] = list(mon_12_combined_nf.mean(axis=1))
mon_12_combined_nf["std"] = list(mon_12_combined_nf.std(axis=1))

mon_6_combined_sp["mean"] = list(mon_6_combined_sp.mean(axis=1))
mon_6_combined_sp["std"] = list(mon_6_combined_sp.std(axis=1))

mon_12_combined_sp["mean"] = list(mon_12_combined_sp.mean(axis=1))
mon_12_combined_sp["std"] = list(mon_12_combined_sp.std(axis=1))

In [276]:
mon_6_zscore_nf = mon_6_combined_nf.copy(deep=True)
mon_12_zscore_nf = mon_12_combined_nf.copy(deep=True)

# Dropping the columns not needed in this z score df
mon_6_zscore_nf.drop(["mean", "std"], inplace=True, axis=1)
mon_12_zscore_nf.drop(["mean", "std"], inplace=True, axis = 1)


mon_6_zscore_sp = mon_6_combined_sp.copy(deep=True)
mon_12_zscore_sp = mon_12_combined_sp.copy(deep=True)

# Dropping the columns not needed in this z score df
mon_6_zscore_sp.drop(["mean", "std"], inplace=True, axis=1)
mon_12_zscore_sp.drop(["mean", "std"], inplace=True, axis = 1)

In [277]:
def zscore(zscore_db, mon_combined, tickers):
    for ticker in tickers:
        zscore_db[ticker] = (mon_combined[ticker]-mon_combined["mean"])/mon_combined["std"]

zscore(mon_6_zscore_nf, mon_6_combined_nf, nf_tickers)
zscore(mon_12_zscore_nf, mon_12_combined_nf, nf_tickers)
zscore(mon_6_zscore_sp, mon_6_combined_sp, sp_tickers)
zscore(mon_12_zscore_sp, mon_12_combined_sp, sp_tickers)

In [278]:

# Calulating Single Momentum Combined Score (C)

def C_score(mon_6_zscore, mon_12_zscore):
    C_score = mon_6_zscore.mul(0.5).add(mon_12_zscore.mul(0.5))
    return C_score

# Single Momentum Combined Scores for all the four cases

# 6 and 12 month Zscores

C_score_nf = C_score(mon_6_zscore_nf, mon_12_zscore_nf)
C_score_sp = C_score(mon_6_zscore_sp, mon_12_zscore_sp)

In [279]:
# Calculating mean and std for the same in order to arrive at zscores

C_score_nf["mean"] = list(C_score_nf.mean(axis=1))
C_score_nf["std"] = list(C_score_nf.std(axis=1))

C_score_sp["mean"] = list(C_score_sp.mean(axis=1))
C_score_sp["std"] = list(C_score_sp.std(axis=1))

/var/folders/29/f90pvtgj0f533wht42wjvwcm0000gn/T/ipykernel_9596/750133191.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  C_score_nf["mean"] = list(C_score_nf.mean(axis=1))
/var/folders/29/f90pvtgj0f533wht42wjvwcm0000gn/T/ipykernel_9596/750133191.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  C_score_nf["std"] = list(C_score_nf.std(axis=1))
/var/folders/29/f90pvtgj0f533wht42wjvwcm0000gn/T/ipykernel_9596/750133191.py:6: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.inse

### Calculating Single Momentum Standardized Z-score

In [280]:
# Calculating Winsorized Z scores for the data below

C_zscore_nf = C_score_nf.copy(deep=True)
C_zscore_sp = C_score_sp.copy(deep=True)

# Dropping the columns not needed in this z score df
C_zscore_nf.drop(["mean", "std"], inplace=True, axis=1)
C_zscore_sp.drop(["mean", "std"], inplace=True, axis = 1)

def winsorized_zscore(C_score, C_zscore, tickers):
    for ticker in tickers:
        C_zscore[ticker] = (C_score[ticker] - C_score["mean"])/C_score["std"]
        C_zscore[ticker] = C_zscore[ticker].apply(lambda x: max(min(x, 3), -3))

winsorized_zscore(C_score_nf, C_zscore_nf, nf_tickers)
winsorized_zscore(C_score_sp, C_zscore_sp, sp_tickers)


In [281]:
# Check if Winsorized or not
C_zscore_nf.describe()

,360ONE,3MINDIA,ABB,ACC,AIAENG,APLAPOLLO,AUBANK,AARTIIND,AAVAS,ABBOTINDIA,...,WESTLIFE,WHIRLPOOL,WIPRO,YESBANK,ZFCVINDIA,ZEEL,ZENSARTECH,ZOMATO,ZYDUSLIFE,ECLERX
count,1062.000000,1858.000000,1858.000000,1858.000000,1858.000000,1858.000000,1601.000000,1858.000000,1293.000000,1858.000000,...,1083.000000,1858.000000,1858.000000,1858.000000,1858.000000,1858.000000,1858.000000,605.000000,1858.000000,1858.000000
mean,0.042040,-0.083199,0.091111,-0.255417,-0.117498,0.270230,-0.019859,-0.004513,-0.196945,-0.007874,...,-0.158802,-0.186517,-0.288899,-0.132576,-0.174677,-0.314603,0.025550,0.333784,-0.117499,-0.175895
std,0.746112,0.923619,1.056478,0.760385,0.789752,0.942546,0.926470,0.934040,0.920362,1.226928,...,0.820323,1.061311,1.006499,0.891062,0.856292,0.761652,1.137339,0.828040,1.083482,1.062885
min,-2.020094,-2.486781,-2.802398,-2.610185,-1.875645,-2.307377,-2.376348,-2.709453,-3.000000,-2.850765,...,-2.047638,-3.000000,-2.398171,-2.265084,-3.000000,-2.600558,-2.513763,-2.061766,-3.000000,-3.000000
25%,-0.493475,-0.730697,-0.683184,-0.755844,-0.678450,-0.324548,-0.716106,-0.627766,-0.823622,-0.934427,...,-0.705671,-0.900701,-1.011102,-0.764878,-0.727327,-0.785384,-0.740888,-0.268208,-0.837432,-0.821911
50%,-0.044123,-0.154998,-0.073757,-0.337955,-0.237986,0.252505,-0.090291,-0.039527,-0.235070,-0.203816,...,-0.173416,-0.252366,-0.426853,-0.217835,-0.207450,-0.339524,-0.103658,0.329033,-0.181210,-0.203061
75%,0.561879,0.432822,0.873476,0.218246,0.326544,0.864615,0.589016,0.599344,0.362034,0.946071,...,0.215486,0.507242,0.347093,0.416550,0.401305,0.106269,0.713515,0.926781,0.563759,0.398150
max,2.368563,3.000000,3.000000,1.892376,3.000000,2.779788,3.000000,3.000000,2.771453,3.000000,...,2.921351,3.000000,3.000000,3.000000,2.397483,2.496966,3.000000,2.375264,3.000000,3.000000


In [282]:
# Check if Winsorized or not
C_zscore_sp.describe()


,AAPL,NVDA,MSFT,GOOG,GOOGL,AMZN,META,BRK-B,LLY,AVGO,...,DAY,WYNN,FMC,PARA,CZR,BBWI,IVZ,BWA,NCLH,AAL
count,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,2359.000000,...,1436.000000,2359.000000,2359.000000,2359.000000,2341.000000,2359.000000,2359.000000,2359.000000,2358.000000,2359.000000
mean,0.276965,0.808596,0.336030,0.169226,0.165970,0.333772,0.290185,-0.128516,0.344081,0.405032,...,0.027106,-0.082984,-0.162749,-0.286587,0.357927,-0.141236,-0.269593,-0.196745,-0.082957,-0.267610
std,1.109408,1.146468,0.780700,0.865632,0.874527,1.036640,0.977078,0.845183,1.269381,0.855146,...,1.077768,0.979980,1.046095,1.128934,1.021544,1.212013,0.887286,0.914238,0.873408,0.915091
min,-2.645685,-2.057415,-1.942185,-2.027079,-2.150803,-2.682077,-2.171179,-3.000000,-2.890257,-2.267950,...,-2.966037,-2.518262,-3.000000,-3.000000,-2.258740,-3.000000,-2.332103,-2.567520,-2.883625,-3.000000
25%,-0.494846,-0.108978,-0.186750,-0.452519,-0.444239,-0.473568,-0.413575,-0.717570,-0.592865,-0.166550,...,-0.796948,-0.729058,-0.800118,-1.086382,-0.385645,-1.001414,-0.875980,-0.891580,-0.685125,-0.896415
50%,0.264932,0.785566,0.352982,0.142833,0.144686,0.376644,0.254542,-0.193322,0.221876,0.325781,...,0.059987,-0.129857,-0.071768,-0.268229,0.282195,-0.124141,-0.327548,-0.125133,-0.086109,-0.247422
75%,1.024737,1.655982,0.846335,0.769680,0.791328,1.089012,0.947665,0.500758,1.277231,0.939648,...,0.761415,0.596837,0.491082,0.409166,1.012117,0.625773,0.204328,0.408553,0.471426,0.341129
max,3.000000,3.000000,2.611881,2.797134,2.946598,3.000000,3.000000,2.411627,3.000000,3.000000,...,3.000000,2.799277,3.000000,3.000000,3.000000,3.000000,2.585355,2.394165,3.000000,2.489069


### Picking Top 10 out of 500

In [283]:
# Take all the present day values of C scores for all the 500 stocks and their tickers and save in another data frame

latest_C_nf = pd.DataFrame(C_zscore_nf.iloc[-1]).reset_index()
latest_C_sp = pd.DataFrame(C_zscore_sp.iloc[-1]).reset_index()

latest_C_nf.columns = ["Symbol", "Winsorized_Score"]
latest_C_sp.columns = ["Symbol", "Winsorized_Score"]

### Results

In [284]:
# Dropping winsorized score for those which the data was insufficient to calculate
nifty_na = latest_C_nf[latest_C_nf["Winsorized_Score"].isna()]

nifty_without_na = latest_C_nf.dropna()

top10_nf = nifty_without_na.sort_values(by="Winsorized_Score", ascending=False).head(10).merge(nf500, how = "left", left_on="Symbol", right_on="Symbol")

print("Top 10 picks based on {} calculations".format(dt.datetime.today().date()))
top10_nf[["Company Name", "Symbol", "Industry", "Winsorized_Score"]]

Top 10 picks based on 2024-08-23 calculations


,Company Name,Symbol,Industry,Winsorized_Score
0,Granules India Ltd.,GRANULES,Healthcare,3.000000
1,PCBL Ltd.,PCBL,Chemicals,3.000000
2,Suzlon Energy Ltd.,SUZLON,Capital Goods,3.000000
3,Glenmark Pharmaceuticals Ltd.,GLENMARK,Healthcare,2.879113
4,Suven Pharmaceuticals Ltd.,SUVENPHAR,Healthcare,2.799543
5,India Cements Ltd.,INDIACEM,Construction Materials,2.758072
6,Bikaji Foods International Ltd.,BIKAJI,Fast Moving Consumer Goods,2.735757
7,Deepak Fertilisers & Petrochemicals Corp. Ltd.,DEEPAKFERT,Chemicals,2.668573
8,Colgate Palmolive (India) Ltd.,COLPAL,Fast Moving Consumer Goods,2.631553
9,Ajanta Pharmaceuticals Ltd.,AJANTPHARM,Healthcare,2.623670


In [285]:
# Dropping winsorized score for those which the data was insufficient to calculate
sp_na = latest_C_sp[latest_C_sp["Winsorized_Score"].isna()]

sp_without_na = latest_C_sp.dropna()

top10_sp = sp_without_na.sort_values(by="Winsorized_Score", ascending=False).head(10).merge(sp500, how = "left", left_on="Symbol", right_on="Symbol")

print("Top 10 picks based on {} calculations".format(dt.datetime.today().date()))
top10_sp[["Shortname", "Symbol", "Industry", "Winsorized_Score", "Marketcap"]]

Top 10 picks based on 2024-08-23 calculations


,Shortname,Symbol,Industry,Winsorized_Score,Marketcap
0,Kellanova,K,Packaged Foods,3.000000,27718688768
1,Lockheed Martin Corporation,LMT,Aerospace & Defense,3.000000,131993124864
2,Iron Mountain Incorporated (Del,IRM,REIT - Specialty,2.442309,32882964480
3,"Universal Health Services, Inc.",UHS,Medical Care Facilities,2.359668,15197266944
4,AbbVie Inc.,ABBV,Drug Manufacturers - General,2.276165,347138785280
5,CBRE Group Inc,CBRE,Real Estate Services,2.254244,34393817088
6,Philip Morris International Inc,PM,Tobacco,2.222694,186575994880
7,"T-Mobile US, Inc.",TMUS,Telecom Services,2.197578,228058824704
8,AFLAC Incorporated,AFL,Insurance - Life,2.003972,58993135616
9,"Regeneron Pharmaceuticals, Inc.",REGN,Biotechnology,1.987479,130665947136
